<a href="https://colab.research.google.com/github/Appleking123456/astro-platform-starter/blob/main/Another_copy_of_Untitled23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import umap.umap_ as umap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings
import json
from datetime import datetime
import os
import random # For reproducibility with random.seed

# Suppress specific UserWarnings from UMAP and KMeans
warnings.filterwarnings("ignore", category=UserWarning)
sns.set_style("whitegrid")

# ===== CONFIGURATION =====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set all seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42) # For any non-PyTorch/Numpy random operations

# Create directory for saving results
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_dir = f"rdt_simulation_results_{timestamp}"
os.makedirs(results_dir, exist_ok=True)
os.makedirs(os.path.join(results_dir, "plots"), exist_ok=True) # Subdir for plots

# Hyperparameters
NUM_EPISODES = 500
MAX_TIME_STEPS_PER_EPISODE = 100
SEQUENCE_LENGTH = 10 # How many past states the agent considers
D_MODEL = 128 # Dimension of internal representation
NUM_HEADS = 8
LEARNING_RATE = 1e-4
SALENCE_LR_MULTIPLIER = 0.1 # A smaller LR for salience prediction to allow why_metric to emerge
MEMORY_SIZE_EPISODIC = 50
MEMORY_SIZE_SEMANTIC = 20

# ===== UTILITY FUNCTIONS =====
def plot_metrics_over_time(history_data, metric_name, title, filename):
    plt.figure(figsize=(12, 6))
    plt.plot([d[metric_name] for d in history_data], label=metric_name)
    plt.title(title)
    plt.xlabel("Simulation Step")
    plt.ylabel(metric_name)
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(results_dir, "plots", filename))
    plt.close()

def plot_umap_embeddings(embeddings, labels, title, filename):
    if len(embeddings) < 2:
        print(f"Not enough data points ({len(embeddings)}) for UMAP plot: {filename}")
        return

    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
    try:
        embedding_2d = reducer.fit_transform(np.array(embeddings))
        plt.figure(figsize=(10, 8))
        if labels is not None:
            scatter = plt.scatter(embedding_2d[:, 0], embedding_2d[:, 1], c=labels, cmap='viridis', s=10, alpha=0.7)
            plt.colorbar(scatter, label="Why Metric Value")
        else:
            plt.scatter(embedding_2d[:, 0], embedding_2d[:, 1], s=10, alpha=0.7)
        plt.title(title)
        plt.xlabel("UMAP Dimension 1")
        plt.ylabel("UMAP Dimension 2")
        plt.grid(True)
        plt.savefig(os.path.join(results_dir, "plots", filename))
        plt.close()
    except Exception as e:
        print(f"UMAP plotting failed for {filename}: {e}")

# ===== RCS AGENT (Observer & Inconsistency Processor) =====
class RCSAgent(nn.Module):
    def __init__(self, state_dim, action_dim, d_model=D_MODEL, num_heads=NUM_HEADS):
        super().__init__()
        self.state_dim = state_dim
        self.d_model = d_model
        self.action_dim = action_dim

        # Input processing with residual connections
        self.input_embedding = nn.Sequential(
            nn.Linear(state_dim, d_model),
            nn.LayerNorm(d_model),
            nn.SiLU(),
            nn.Dropout(0.1)
        ).to(device)

        # Multi-scale feature extraction for the *last* state in the sequence
        # Assuming state_dim is large enough for meaningful conv features
        self.local_features = nn.Conv1d(1, d_model//4, kernel_size=3, padding=1).to(device)
        self.global_features = nn.Conv1d(1, d_model//4, kernel_size=7, padding=3).to(device)

        # Calculate conv_feature_dim dynamically based on an example input
        # This is a bit tricky; a more robust way is to pass a dummy tensor
        # For simplicity, let's assume the conv output will be flattened and projected to d_model
        # The output size of Conv1d with padding='same' is the input size. So (batch, 1, state_dim) -> (batch, d_model//4, state_dim)
        # Flattening (d_model//4 * state_dim)

        # We need the actual state_dim to be divisible by the kernel for consistent conv output size.
        # For a general solution, let's just make it a large linear layer that projects it.
        # This might need adjustment if state_dim is too large for these convs.
        self.conv_proj = nn.Linear(state_dim * (d_model // 4) * 2, d_model).to(device) # Placeholder size, adjust as needed

        # Transformer encoder for sequence processing
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=d_model * 4,
            dropout=0.1,
            batch_first=True,
            activation='gelu',
            device=device
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=4).to(device)

        # Attention mechanism for memory
        self.memory_attention = nn.MultiheadAttention(
            embed_dim=d_model,
            num_heads=4,
            batch_first=True,
            device=device
        )

        # Inconsistency Detector - Core RDT component
        self.inconsistency_detector = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.SiLU(),
            nn.Linear(64, 1),
            nn.Sigmoid() # Output a score between 0 and 1 (0: consistent, 1: highly inconsistent)
        ).to(device)

        # Meta-Cognition / Self-Referential Unit (Why Processor) - Core RDT component
        # Takes current state embedding and inconsistency to update an internal "why_accumulator"
        self.why_processor = nn.LSTMCell(d_model + 1, d_model // 2).to(device)
        # Registered as buffers to persist across forward passes within an episode, but reset per episode
        self.register_buffer('why_hidden', torch.zeros(1, d_model // 2, device=device))
        self.register_buffer('why_cell', torch.zeros(1, d_model // 2, device=device))
        self.why_output_layer = nn.Linear(d_model // 2, 1).to(device) # Project to a single 'why_metric'

        # Exploration Policy (formerly Actor) - for probing the environment
        self.exploration_policy = nn.Sequential(
            nn.Linear(d_model + 1 + d_model // 2, 128), # Input: state_embedding, inconsistency, why_hidden
            nn.SiLU(),
            nn.Linear(128, 64),
            nn.SiLU(),
            nn.Linear(64, action_dim),
            nn.Tanh() # Scale actions between -1 and 1
        ).to(device)

        # Salience Predictor (formerly Critic) - predicts info gain potential
        self.salience_predictor = nn.Sequential(
            nn.Linear(d_model + 1, 128), # Input: state_embedding + inconsistency_score
            nn.SiLU(),
            nn.Linear(128, 64),
            nn.SiLU(),
            nn.Linear(64, 1)
        ).to(device)

        # Episodic and Semantic Memory
        self.register_buffer('episodic_memory', torch.zeros(MEMORY_SIZE_EPISODIC, d_model, device=device))
        self.register_buffer('semantic_memory', torch.zeros(MEMORY_SIZE_SEMANTIC, d_model, device=device))
        self.register_buffer('episodic_ptr', torch.tensor(0, device=device, dtype=torch.long))
        self.register_buffer('semantic_ptr', torch.tensor(0, device=device, dtype=torch.long))

        # Meta-learning adaptation rate (kept as before)
        self.adaptation_rate = nn.Parameter(torch.tensor(0.01, device=device))

        self._init_weights()

    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
            elif isinstance(module, (nn.LayerNorm, nn.BatchNorm1d)):
                nn.init.constant_(module.bias, 0)
                nn.init.constant_(module.weight, 1.0)

    def update_memory(self, state_embedding, salience_score, inconsistency_score, why_metric):
        # Only store if salience is above a threshold, or if inconsistency/why is high
        if salience_score.item() > 0.6 or inconsistency_score.item() > 0.7 or why_metric.item() > 0.5:
            self.episodic_memory.data[self.episodic_ptr] = state_embedding.detach().data
            self.episodic_ptr.data = (self.episodic_ptr.data + 1) % self.episodic_memory.size(0)

            # Semantic memory update for highly significant/inconsistent states or high why
            if salience_score.item() > 0.8 and inconsistency_score.item() > 0.8 and why_metric.item() > 0.7:
                 self.semantic_memory.data[self.semantic_ptr] = state_embedding.detach().data
                 self.semantic_ptr.data = (self.semantic_ptr.data + 1) % self.semantic_memory.size(0)

    def reset_why_state(self):
        """Resets the internal state of the why_processor for a new episode."""
        self.why_hidden = torch.zeros(1, self.d_model // 2, device=device)
        self.why_cell = torch.zeros(1, self.d_model // 2, device=device)

    def forward(self, state_sequence):
        batch_size = state_sequence.size(0)
        seq_len = state_sequence.size(1)

        # Multi-scale feature extraction on the *last* state
        last_state = state_sequence[:, -1, :].unsqueeze(1) # (batch_size, 1, state_dim)

        # Handle cases where state_dim might be too small for Conv1d kernel
        if self.state_dim < 7: # Smallest kernel size used in conv
             # Fallback: simpler feature extraction if conv is not applicable
            local_feat = F.adaptive_avg_pool1d(last_state, self.d_model // 4).transpose(1,2)
            global_feat = F.adaptive_max_pool1d(last_state, self.d_model // 4).transpose(1,2)
        else:
            local_feat = self.local_features(last_state).transpose(1, 2)
            global_feat = self.global_features(last_state).transpose(1, 2)

        combined_conv_features = torch.cat([local_feat.flatten(start_dim=1), global_feat.flatten(start_dim=1)], dim=1)
        projected_conv = self.conv_proj(combined_conv_features) # Project to d_model

        # Sequence embedding and positional encoding
        x_seq_embedding = self.input_embedding(state_sequence.float())
        pos_encoding = torch.zeros(seq_len, self.d_model, device=device)
        position = torch.arange(0, seq_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, self.d_model, 2).float() * -(np.log(10000.0) / self.d_model))
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)
        x = x_seq_embedding + pos_encoding.unsqueeze(0)

        # Transformer processing
        memory = self.transformer(x)
        state_embedding = memory[:, -1, :] # Embedding of the last element in sequence

        # Combine transformer output with conv features
        state_embedding = state_embedding + projected_conv

        # Memory attention mechanism
        current_episodic_memory = self.episodic_memory[:self.episodic_ptr] # Only use filled part
        current_semantic_memory = self.semantic_memory[:self.semantic_ptr]

        if current_episodic_memory.numel() > 0:
            attended_episodic_memory, _ = self.memory_attention(
                state_embedding.unsqueeze(1),
                current_episodic_memory.unsqueeze(0).expand(batch_size, -1, -1),
                current_episodic_memory.unsqueeze(0).expand(batch_size, -1, -1)
            )
            state_embedding = state_embedding + 0.1 * attended_episodic_memory.squeeze(1)

        if current_semantic_memory.numel() > 0:
            attended_semantic_memory, _ = self.memory_attention(
                state_embedding.unsqueeze(1),
                current_semantic_memory.unsqueeze(0).expand(batch_size, -1, -1),
                current_semantic_memory.unsqueeze(0).expand(batch_size, -1, -1)
            )
            state_embedding = state_embedding + 0.2 * attended_semantic_memory.squeeze(1) # Stronger influence for semantic

        # Calculate Inconsistency Score (Core RDT)
        inconsistency_score = self.inconsistency_detector(state_embedding)

        # Update Why Accumulator (Core RDT)
        # LSTMCell expects (input, (h, c)) where input is (batch_size, input_size)
        # and h, c are (batch_size, hidden_size)
        why_input_combined = torch.cat([state_embedding, inconsistency_score], dim=-1)

        # Unpack, process, and pack LSTM states, handling batch_size=1
        h_in = self.why_hidden.squeeze(0) # (hidden_size,)
        c_in = self.why_cell.squeeze(0)   # (hidden_size,)

        h_out, c_out = self.why_processor(why_input_combined.squeeze(0), (h_in, c_in))

        self.why_hidden = h_out.unsqueeze(0) # (1, hidden_size)
        self.why_cell = c_out.unsqueeze(0)   # (1, hidden_size)

        why_metric = torch.sigmoid(self.why_output_layer(self.why_hidden)) # Sigmoid to keep between 0 and 1

        # Exploration Policy (action generation)
        exploration_input = torch.cat([state_embedding, inconsistency_score, self.why_hidden], dim=-1)
        exploration_action = self.exploration_policy(exploration_input)

        # Salience Predictor
        salience = self.salience_predictor(torch.cat([state_embedding, inconsistency_score], dim=-1)) # Predicts how 'interesting' or 'informative' this state is

        # Update Memory (using combined criteria)
        # Pass batch_size 1, as this is typically run step-by-step
        self.update_memory(state_embedding.squeeze(0), salience.squeeze(0), inconsistency_score.squeeze(0), why_metric.squeeze(0))

        return exploration_action, salience, state_embedding, inconsistency_score, why_metric

# ===== DYNAMIC TENSOR (5D Substrate Representation) =====
class DynamicTensor(nn.Module):
    def __init__(self, size=(4,4,4)):
        super().__init__()
        self.initial_size = size
        self.history = []
        self.phase_history = []

        # Use register_parameter for gradients, register_buffer for non-gradient tensors
        self._tensor_data = nn.Parameter(self._initialize_tensor(size))
        self._entanglement_level = nn.Parameter(torch.tensor(0.5, device=device, requires_grad=True))
        self._chaoticity_index = nn.Parameter(torch.tensor(0.1, device=device, requires_grad=True))

        # Enhanced learnable evolution parameters
        self.evolution_rate = nn.Parameter(torch.tensor(0.08, device=device))
        self.syntropic_strength = nn.Parameter(torch.tensor(0.15, device=device))
        self.quantum_coupling = nn.Parameter(torch.tensor(0.03, device=device))

        # Phase transition parameters
        self.critical_entanglement = nn.Parameter(torch.tensor(0.618, device=device))
        self.phase_transition_sharpness = nn.Parameter(torch.tensor(5.0, device=device))

    def _initialize_tensor(self, size):
        base_tensor = torch.randn(size, device=device)
        pattern = torch.zeros(size, device=device)
        for i in range(size[0]):
            for j in range(size[1]):
                x = torch.linspace(0, 2*np.pi, size[2]).to(device)
                structure = (0.3 * torch.sin(x + i * np.pi/4) +
                           0.2 * torch.cos(2*x + j * np.pi/3) +
                           0.1 * torch.sin(3*x))
                pattern[i, j] = structure
        return base_tensor + pattern

    def compute_phase_order(self, tensor_data):
        """Compute an order parameter for phase transitions (e.g., spatial correlation)"""
        if tensor_data.numel() < 2:
            return torch.tensor(0.0, device=device)

        flat_data = tensor_data.flatten()
        correlations = []

        for shift in [1, 2]: # Check correlations with small shifts
            if len(flat_data) > shift:
                x_shifted = flat_data[:-shift]
                y_unshifted = flat_data[shift:]

                mean_x = x_shifted.mean()
                mean_y = y_unshifted.mean()

                cov_xy = ((x_shifted - mean_x) * (y_unshifted - mean_y)).mean()
                std_x = x_shifted.std()
                std_y = y_unshifted.std()

                if std_x > 1e-8 and std_y > 1e-8:
                    corr = cov_xy / (std_x * std_y)
                else:
                    corr = torch.tensor(0.0, device=device)

                correlations.append(corr)

        return torch.stack(correlations).mean() if correlations else torch.tensor(0.0, device=device)

    def forward(self, current_tensor_data, current_entanglement_level, current_chaoticity_index):
        # Ensure parameters are correctly used for gradient flow
        # current_entanglement_level and current_chaoticity_index are now directly updated by the agent's actions
        # and fed back into this forward pass.

        # Enhanced multi-scale evolution
        noise = torch.randn_like(current_tensor_data) * self.evolution_rate
        evolved_data = current_tensor_data + noise

        # Phase transition dynamics based on agent's action-influenced entanglement
        phase_order = self.compute_phase_order(evolved_data)
        entanglement_diff = current_entanglement_level - self.critical_entanglement
        phase_transition_factor = torch.tanh(
            self.phase_transition_sharpness * entanglement_diff
        )

        # Enhanced syntropic dynamics
        std_val = evolved_data.std() + 1e-8
        mean_val = evolved_data.mean()

        # Syntropic pull towards a more organized state, influenced by phase transition
        syntropic_pull = self.syntropic_strength * (
            (mean_val + std_val) * (1 + phase_transition_factor * 0.5) - evolved_data
        )

        # Multi-dimensional quantum-inspired dynamics
        quantum_influence = torch.zeros_like(evolved_data)
        # Apply quantum influence only if the tensor is large enough for FFT and to avoid errors
        if evolved_data.numel() > 1 and evolved_data.dim() >= 2:
            try:
                # Take the first 2D slice if it's a 3D tensor, or use as is if 2D
                fft_input = evolved_data[0] if evolved_data.dim() > 2 else evolved_data
                if fft_input.dim() < 2: # If it's 1D after slicing (e.g., initial_size[0]=1, but [1],[2] are larger)
                     # Reshape to 2D for FFT2 if possible
                     if fft_input.numel() > 1:
                         side_len = int(np.sqrt(fft_input.numel()))
                         if side_len * side_len == fft_input.numel():
                             fft_input = fft_input.reshape(side_len, side_len)
                         else: # Pad if not square
                             pad_size = side_len*side_len - fft_input.numel() if side_len*side_len > fft_input.numel() else 0
                             fft_input = F.pad(fft_input.flatten(), (0, pad_size)).reshape(side_len, side_len)
                     else: # Single element
                         fft_input = torch.tensor([[fft_input.item()]], device=device) # Make it 2D

                fft_data = torch.fft.fft2(fft_input)
                phase_spectrum = torch.angle(fft_data)
                quantum_influence_scalar = self.quantum_coupling * torch.cos(phase_spectrum.real).mean()
                quantum_influence = quantum_influence_scalar * torch.randn_like(evolved_data)
            except Exception as e:
                # Fallback if FFT fails (e.g. for odd dimensions)
                # print(f"Warning: FFT failed for DynamicTensor, skipping quantum influence: {e}")
                quantum_influence = torch.randn_like(evolved_data) * self.quantum_coupling * 0.1 # Small random influence

        # Combine all evolution mechanisms
        evolved_data = (
            (1 - current_chaoticity_index) * evolved_data + # Less chaoticity -> more regular evolution
            current_chaoticity_index * syntropic_pull +    # More chaoticity -> stronger syntropic pull
            quantum_influence +
            0.05 * phase_transition_factor * torch.randn_like(evolved_data) # Randomness influenced by phase
        )

        # Entanglement-driven stabilization/coherence
        entanglement_effect = torch.sigmoid((current_entanglement_level - 0.5) * 8) # Stronger effect around 0.5
        evolved_data = (
            (1 - entanglement_effect) * evolved_data +
            entanglement_effect * (evolved_data.mean() + 0.2 * evolved_data) # Pull towards mean for higher entanglement
        )

        # Store extended history - use detached values for history
        if len(self.history) > 150: # Cap history length
            self.history.pop(0)
        if len(self.phase_history) > 100:
            self.phase_history.pop(0)

        self.history.append(evolved_data.detach().cpu().numpy())
        self.phase_history.append(phase_order.detach().cpu().numpy())

        return evolved_data

    def get_current_tensor_data(self):
        return self._tensor_data

    def get_entanglement_level(self):
        return self._entanglement_level

    def get_chaoticity_index(self):
        return self._chaoticity_index

    def set_current_tensor_data(self, new_data):
        # Use .data to update nn.Parameter without breaking graph for other parts
        self._tensor_data.data = new_data.data

    def set_entanglement_level(self, new_level):
        # Clamp to reasonable bounds to prevent instability
        self._entanglement_level.data = torch.clamp(new_level, 0.01, 0.99).data

    def set_chaoticity_index(self, new_index):
        # Clamp to reasonable bounds
        self._chaoticity_index.data = torch.clamp(new_index, 0.001, 0.8).data

    def metrics(self):
        t_data = self._tensor_data.detach()
        ent_level = self._entanglement_level.detach().item()
        cha_index = self._chaoticity_index.detach().item()

        t_flat = t_data.flatten()

        # Ensure t_normalized_for_entropy is strictly positive for log
        min_val_for_softmax = 1e-8
        t_normalized = F.softmax(t_flat, dim=0) + min_val_for_softmax
        entropy_val = -(t_normalized * torch.log(t_normalized)).sum().item()

        variance = t_data.var().item()
        complexity = entropy_val * variance # Simple complexity metric

        # Fractal dimension estimation (Box-counting method - simplified)
        fractal_dim = 1.0 # Default if calculation fails
        try:
            if len(self.history) > 10:
                recent_history_flat = np.array(self.history[-10:]).flatten()
                if len(recent_history_flat) > 0 and np.std(recent_history_flat) > 1e-8:
                    # Normalize for box-counting if range is too small
                    min_h, max_h = np.min(recent_history_flat), np.max(recent_history_flat)
                    if max_h - min_h < 1e-8: # Avoid division by zero
                        normalized_history = recent_history_flat
                    else:
                        normalized_history = (recent_history_flat - min_h) / (max_h - min_h)

                    # Create a 1D grid for box counting
                    num_bins = 10
                    bins = np.linspace(0, 1, num_bins + 1)
                    counts = []
                    for i in range(num_bins):
                        count = np.sum((normalized_history >= bins[i]) & (normalized_history < bins[i+1]))
                        if count > 0: counts.append(count)

                    if len(counts) > 1:
                        # Simplified approach: log-log plot of count vs box size
                        # Use a range of effective "box sizes" relative to the normalized range
                        scales = np.array([1/num_bins, 2/num_bins, 3/num_bins]) # Example scales
                        log_scales = np.log(scales)

                        # Assuming counts represent occupancy for these scales (rough approx)
                        log_counts = np.log(np.array([sum(counts)] * len(scales))) # Placeholder: total count for all boxes
                        if len(log_scales) > 1 and np.std(log_counts) > 1e-8:
                            slope, _ = np.polyfit(log_scales, log_counts, 1)
                            fractal_dim = -slope if not np.isnan(slope) else 1.0
        except Exception:
            pass # Keep default if any error in fractal dim calculation

        # Phase coherence (higher is more coherent)
        phase_coherence = 0.0
        if len(self.phase_history) > 5:
            recent_phases = np.array(self.phase_history[-5:])
            mean_abs_phase = np.mean(np.abs(recent_phases)) + 1e-8
            if mean_abs_phase > 0:
                phase_coherence = 1.0 - np.std(recent_phases) / mean_abs_phase
            phase_coherence = np.clip(phase_coherence, 0.0, 1.0) # Clamp between 0 and 1

        return {
            'entropy': entropy_val,
            'entanglement_value': ent_level,
            'chaoticity_value': cha_index,
            'variance': variance,
            'complexity': complexity,
            'fractal_dimension': fractal_dim,
            'phase_coherence': phase_coherence
        }

    def reset(self):
        self._tensor_data.data = self._initialize_tensor(self.initial_size).data
        self._entanglement_level.data = torch.tensor(0.5, device=device).data
        self._chaoticity_index.data = torch.tensor(0.1, device=device).data
        self.history.clear()
        self.phase_history.clear()

# ===== CONSCIOUSNESS KERNEL (4D Projection) =====
class ConsciousnessKernel(nn.Module):
    def __init__(self, target_spectrum_size=4):
        super().__init__()
        self.spectrum_size = target_spectrum_size

        # Learnable frequencies (gamma_freq added)
        self.delta_freq = nn.Parameter(torch.tensor(0.5, device=device))
        self.theta_freq = nn.Parameter(torch.tensor(0.8, device=device))
        self.alpha_freq = nn.Parameter(torch.tensor(1.2, device=device))
        self.beta_freq = nn.Parameter(torch.tensor(1.8, device=device))
        self.gamma_freq = nn.Parameter(torch.tensor(2.5, device=device))

        # Dynamic weights with attention mechanism
        self.attention_weights = nn.Parameter(torch.ones(5, target_spectrum_size, device=device))

        # Neural entanglement processor
        self.entanglement_processor = nn.Sequential(
            nn.Linear(1, 32),
            nn.SiLU(),
            nn.Dropout(0.1),
            nn.Linear(32, 64),
            nn.SiLU(),
            nn.Linear(64, target_spectrum_size * 5), # Output size for modulation of 5 frequencies across spectrum_size
            nn.Tanh() # Scale to [-1, 1] for modulation
        ).to(device)

    def forward(self, tensor_data, entanglement_level):
        # Normalize tensor_data to prevent large numbers from destabilizing Bessel functions
        # This acts as the "4D projection filter" - normalizing the bulk data
        norm_tensor_data = (tensor_data - tensor_data.min()) / (tensor_data.max() - tensor_data.min() + 1e-8) if tensor_data.numel() > 1 else tensor_data

        # Calculate spatial norms based on the normalized tensor data
        spatial_norms = []
        if norm_tensor_data.dim() == 0: # Handle scalar tensor
            spatial_norms.append(norm_tensor_data.unsqueeze(0))
        elif norm_tensor_data.numel() == 1: # Handle single element tensor
            spatial_norms.append(norm_tensor_data.unsqueeze(0))
        else:
            # Aggregate information across dimensions to get spatial components
            for dim_idx in range(len(norm_tensor_data.shape)):
                dims_to_reduce = [i for i in range(len(norm_tensor_data.shape)) if i != dim_idx]
                spatial_norms.append(torch.norm(norm_tensor_data, dim=dims_to_reduce).mean().unsqueeze(0))

        if not spatial_norms: # Fallback for edge cases
            spatial_norms.append(torch.tensor([1.0], device=device))

        # Ensure spatial_norms has enough elements for spectrum_size
        if len(spatial_norms) < self.spectrum_size:
            # Pad by repeating the last element or similar strategy
            spatial_norms_extended = spatial_norms + [spatial_norms[-1]] * (self.spectrum_size - len(spatial_norms))
            spatial_norms = spatial_norms_extended[:self.spectrum_size] # Trim if too long

        # R_total can be a general measure of the "size" or "magnitude" of the tensor state
        r_total = torch.norm(norm_tensor_data) # Total magnitude of the normalized tensor

        # Process entanglement to get modulation factors
        ent_processed_raw = self.entanglement_processor(entanglement_level.unsqueeze(0))
        ent_modulation_factors = ent_processed_raw.reshape(5, self.spectrum_size) * 0.5 + 0.5 # Scale to [0, 1] for multiplication

        # Generate multi-frequency spectrum components using Bessel functions for wave-like patterns
        frequencies = [self.delta_freq, self.theta_freq, self.alpha_freq, self.beta_freq, self.gamma_freq]
        spectrum_components = []

        for freq_idx, freq in enumerate(frequencies):
            components = []
            for i in range(self.spectrum_size):
                phase = (i / self.spectrum_size) * 2 * np.pi # Phase across the spectrum
                current_spatial_component = spatial_norms[i].squeeze()

                # The argument for Bessel function combines frequency, entanglement modulation,
                # and spatial component, representing complex wave interactions
                arg = freq * ent_modulation_factors[freq_idx, i] * (current_spatial_component + r_total * 0.1)

                # Using Bessel functions to generate wave-like patterns (j0, j1)
                # Adding small phase shifts for diversity
                if freq_idx == 0: value = torch.special.bessel_j0(arg) * torch.cos(torch.tensor(phase, device=device))
                elif freq_idx == 1: value = torch.special.bessel_j1(arg) * torch.sin(torch.tensor(phase, device=device))
                elif freq_idx == 2: value = torch.special.bessel_j0(arg + np.pi/4) * torch.cos(torch.tensor(phase + np.pi/8, device=device))
                elif freq_idx == 3: value = torch.special.bessel_j1(arg + np.pi/3) * torch.sin(torch.tensor(phase + np.pi/6, device=device))
                else: value = torch.special.bessel_j0(arg + 0.5) * torch.cos(torch.tensor(phase + np.pi/4, device=device))

                components.append(value)

            spectrum_components.append(torch.stack(components))

        # Attention-weighted combination of frequency components
        combined_spectrum = torch.zeros(self.spectrum_size, device=device)
        attn_weights_norm = F.softmax(self.attention_weights, dim=0)

        for freq_idx, spectrum in enumerate(spectrum_components):
            combined_spectrum += attn_weights_norm[freq_idx] * spectrum

        # Normalize the final spectrum to [0, 1] for consistent output scale
        combined_spectrum_min = combined_spectrum.min()
        combined_spectrum_max = combined_spectrum.max()
        if combined_spectrum_max - combined_spectrum_min > 1e-8:
            normalized_spectrum = (combined_spectrum - combined_spectrum_min) / (combined_spectrum_max - combined_spectrum_min)
        else:
            normalized_spectrum = torch.zeros_like(combined_spectrum) # All zeros if min=max

        return normalized_spectrum

# ===== RDT ENVIRONMENT =====
class RDTEnvironment:
    def __init__(self, device, tensor_size=(4,4,4)):
        self.device = device
        self.tensor_size = tensor_size
        self.tensor = DynamicTensor(size=self.tensor_size).to(device)
        self.kernel = ConsciousnessKernel(target_spectrum_size=self.tensor_size[-1]).to(device) # Spectrum size matches last dim of tensor

        self.time = 0
        self.max_time = MAX_TIME_STEPS_PER_EPISODE
        self.episode_count = 0
        self.prev_action = None

    def step(self, action):
        action = action.squeeze()

        # Action processing with momentum
        momentum_factor = 0.9
        if self.prev_action is not None:
            action = momentum_factor * self.prev_action + (1 - momentum_factor) * action
        self.prev_action = action.detach().clone()

        # Scale actions. The agent's exploration policy determines magnitude
        scaled_action = action * 0.1 # A small fixed scaling for now, agent learns magnitude

        # Update entanglement and chaoticity (driven by agent's exploration action)
        new_entanglement_level = self.tensor.get_entanglement_level() + scaled_action[0]
        new_chaoticity_index = self.tensor.get_chaoticity_index() + scaled_action[1]
        self.tensor.set_entanglement_level(new_entanglement_level)
        self.tensor.set_chaoticity_index(new_chaoticity_index)

        current_tensor_data = self.tensor.get_current_tensor_data()
        current_entanglement_level = self.tensor.get_entanglement_level()
        current_chaoticity_index = self.tensor.get_chaoticity_index()

        # Evolve the tensor (5D substrate)
        evolved_tensor_data = self.tensor(
            current_tensor_data,
            current_entanglement_level,
            current_chaoticity_index
        )
        self.tensor.set_current_tensor_data(evolved_tensor_data)

        # Project to consciousness spectrum (4D observation)
        projected_spectrum = self.kernel(
            self.tensor.get_current_tensor_data(),
            self.tensor.get_entanglement_level()
        )

        # "Pseudo-Reward" or "Information Gain Signal" for the agent's salience predictor
        # This encourages the agent to explore and engage with complex, coherent states.
        metrics = self.tensor.metrics()

        pseudo_reward = (
            metrics['complexity'] * 0.05 +          # Encourage high complexity
            metrics['phase_coherence'] * 0.1 +       # Encourage coherent patterns
            metrics['fractal_dimension'] * 0.02 -    # Encourage structured complexity, not just chaos
            metrics['entropy'] * 0.01                # Small penalty for pure randomness
        )
        pseudo_reward = torch.tensor(pseudo_reward, device=self.device, dtype=torch.float32)

        # Construct the new observation state for the agent
        tensor_metrics_tensor = torch.tensor([
            metrics['entropy'], metrics['entanglement_value'], metrics['chaoticity_value'],
            metrics['variance'], metrics['complexity'], metrics['fractal_dimension'],
            metrics['phase_coherence']
        ], device=self.device, dtype=torch.float32)

        # Combine spectrum, tensor metrics, and agent-controlled parameters (entanglement, chaoticity)
        new_state = torch.cat([
            projected_spectrum,
            tensor_metrics_tensor,
            current_entanglement_level.unsqueeze(0),
            current_chaoticity_index.unsqueeze(0)
        ]).detach()

        self.time += 1
        done = self.time >= self.max_time

        return new_state, pseudo_reward, done, metrics, projected_spectrum

    def reset(self):
        self.tensor.reset()
        self.time = 0
        self.episode_count += 1
        self.prev_action = None

        # Generate initial state for the agent
        initial_metrics = self.tensor.metrics()
        initial_projected_spectrum = self.kernel(
            self.tensor.get_current_tensor_data(),
            self.tensor.get_entanglement_level()
        )
        initial_state = torch.cat([
            initial_projected_spectrum,
            torch.tensor([
                initial_metrics['entropy'], initial_metrics['entanglement_value'], initial_metrics['chaoticity_value'],
                initial_metrics['variance'], initial_metrics['complexity'], initial_metrics['fractal_dimension'],
                initial_metrics['phase_coherence']
            ], device=self.device, dtype=torch.float32),
            self.tensor.get_entanglement_level().unsqueeze(0),
            self.tensor.get_chaoticity_index().unsqueeze(0)
        ]).detach()
        return initial_state

# ===== MAIN SIMULATION LOOP =====
if __name__ == "__main__":
    # Determine dynamic tensor size and corresponding state_dim for agent
    TENSOR_INITIAL_SIZE = (4, 4, 4) # Example 3D tensor
    STATE_DIM_FOR_AGENT = TENSOR_INITIAL_SIZE[-1] + 7 + 2 # Spectrum size + 7 tensor metrics + 2 agent-controlled params

    agent = RCSAgent(STATE_DIM_FOR_AGENT, action_dim=2).to(device)
    env = RDTEnvironment(device, tensor_size=TENSOR_INITIAL_SIZE)

    # Optimizer for the agent's learnable parameters
    # We are primarily optimizing the salience predictor's ability to model "interestingness"
    # and the exploration policy to generate useful probes.
    optimizer = torch.optim.Adam(agent.parameters(), lr=LEARNING_RATE)

    # Data collection for analysis
    all_metrics_history = [] # Stores all detailed metrics for each step
    all_state_embeddings = [] # For UMAP
    all_why_metrics = [] # For UMAP labels and time series

    print(f"Starting RDT Simulation on device: {device}")
    print(f"Tensor size: {TENSOR_INITIAL_SIZE}, Agent state_dim: {STATE_DIM_FOR_AGENT}")

    for episode in tqdm(range(NUM_EPISODES), desc="Episodes"):
        initial_state = env.reset()
        agent.reset_why_state() # Reset why_processor internal states for each episode

        # Initialize state sequence buffer for the start of the episode
        # Fill with initial state
        state_sequence_buffer = [initial_state.clone()] * SEQUENCE_LENGTH

        episode_why_metrics = []
        episode_inconsistency_scores = []
        episode_salience_predictions = []
        episode_pseudo_rewards = []

        for t in range(MAX_TIME_STEPS_PER_EPISODE):
            current_state_sequence = torch.stack(state_sequence_buffer[-SEQUENCE_LENGTH:]).unsqueeze(0) # Batch size 1

            # Agent processes
            exploration_action, salience_pred, state_embedding, inconsistency_score, why_metric = agent(current_state_sequence)

            # Environment steps
            new_state, pseudo_reward, done, tensor_metrics, projected_spectrum = env.step(exploration_action.squeeze(0)) # Squeeze action for env

            # Store data for analysis
            all_state_embeddings.append(state_embedding.squeeze(0).detach().cpu().numpy())
            all_why_metrics.append(why_metric.squeeze().item())

            episode_why_metrics.append(why_metric.squeeze().item())
            episode_inconsistency_scores.append(inconsistency_score.squeeze().item())
            episode_salience_predictions.append(salience_pred.squeeze().item())
            episode_pseudo_rewards.append(pseudo_reward.item())

            # Store all detailed metrics for this step
            all_metrics_history.append({
                'episode': episode,
                'time_step': t,
                'inconsistency_score': inconsistency_score.squeeze().item(),
                'why_metric': why_metric.squeeze().item(),
                'salience_predicted': salience_pred.squeeze().item(),
                'pseudo_reward_env': pseudo_reward.item(),
                'entanglement_level': env.tensor.get_entanglement_level().item(),
                'chaoticity_index': env.tensor.get_chaoticity_index().item(),
                'tensor_complexity': tensor_metrics['complexity'],
                'tensor_phase_coherence': tensor_metrics['phase_coherence'],
                'tensor_fractal_dimension': tensor_metrics['fractal_dimension'],
                'tensor_entropy': tensor_metrics['entropy'],
                'tensor_variance': tensor_metrics['variance']
            })

            # --- Agent Internal Adaptation / "Learning" ---
            # The 'loss' is for training the salience predictor to better estimate the environment's pseudo_reward.
            # This implicitly trains the agent to seek out "interesting" states.
            salience_loss = F.mse_loss(salience_pred.squeeze(), pseudo_reward)

            # Backpropagation and optimization
            optimizer.zero_grad()
            salience_loss.backward(retain_graph=False) # No need to retain graph if only one loss

            # Apply adaptation rate to gradients
            # This allows the agent to control its own learning speed based on internal state
            for param_group in optimizer.param_groups:
                for p in param_group['params']:
                    if p.grad is not None:
                        p.grad.data.mul_(agent.adaptation_rate)

            optimizer.step()

            # Update state sequence buffer for next step
            state_sequence_buffer.append(new_state.clone())
            if len(state_sequence_buffer) > SEQUENCE_LENGTH:
                state_sequence_buffer.pop(0)

            if done:
                break

        # Episode summary
        avg_why_episode = np.mean(episode_why_metrics)
        avg_inconsistency_episode = np.mean(episode_inconsistency_scores)
        tqdm.write(f"Episode {episode+1}/{NUM_EPISODES} | "
                   f"Avg Why: {avg_why_episode:.4f} | "
                   f"Avg Inconsistency: {avg_inconsistency_episode:.4f} | "
                   f"Last Entanglement: {env.tensor.get_entanglement_level().item():.3f} | "
                   f"Last Chaoticity: {env.tensor.get_chaoticity_index().item():.3f}")

    print("\nSimulation complete. Generating plots and saving results...")

    # ===== POST-SIMULATION ANALYSIS & PLOTTING =====

    # Save all detailed metrics to JSON
    results_json_path = os.path.join(results_dir, "simulation_metrics_history.json")
    with open(results_json_path, "w") as f:
        json.dump(all_metrics_history, f, indent=4)
    print(f"Detailed metrics saved to {results_json_path}")

    # Plot time series of key metrics
    plot_metrics_over_time(all_metrics_history, 'why_metric', 'Why Metric Over Time', 'why_metric_time_series.png')
    plot_metrics_over_time(all_metrics_history, 'inconsistency_score', 'Inconsistency Score Over Time', 'inconsistency_score_time_series.png')
    plot_metrics_over_time(all_metrics_history, 'salience_predicted', 'Salience Predicted by Agent Over Time', 'salience_predicted_time_series.png')
    plot_metrics_over_time(all_metrics_history, 'pseudo_reward_env', 'Environmental Pseudo-Reward Over Time', 'pseudo_reward_env_time_series.png')
    plot_metrics_over_time(all_metrics_history, 'entanglement_level', 'Entanglement Level Over Time (Controlled by Agent)', 'entanglement_level_time_series.png')
    plot_metrics_over_time(all_metrics_history, 'chaoticity_index', 'Chaoticity Index Over Time (Controlled by Agent)', 'chaoticity_index_time_series.png')
    plot_metrics_over_time(all_metrics_history, 'tensor_complexity', 'Tensor Complexity Over Time', 'tensor_complexity_time_series.png')
    plot_metrics_over_time(all_metrics_history, 'tensor_phase_coherence', 'Tensor Phase Coherence Over Time', 'tensor_phase_coherence_time_series.png')

    # UMAP plotting for state embeddings
    plot_umap_embeddings(all_state_embeddings, all_why_metrics, 'UMAP of Agent State Embeddings (Colored by Why Metric)', 'umap_state_embeddings_why.png')

    # Optional: UMAP on why_processor hidden states if desired for deeper analysis
    # Need to collect these explicitly in the loop if you want to plot them

    print(f"Plots saved to {os.path.join(results_dir, 'plots')}")
    print("Simulation finished. Analyze the results to look for patterns of recursive breach!")

Starting RDT Simulation on device: cpu
Tensor size: (4, 4, 4), Agent state_dim: 13


Episodes:   0%|          | 1/500 [00:03<27:35,  3.32s/it]

Episode 1/500 | Avg Why: 0.6214 | Avg Inconsistency: 0.5649 | Last Entanglement: 0.990 | Last Chaoticity: 0.800


Episodes:   0%|          | 2/500 [00:07<31:43,  3.82s/it]

Episode 2/500 | Avg Why: 0.5852 | Avg Inconsistency: 0.4542 | Last Entanglement: 0.990 | Last Chaoticity: 0.800


...

Episodes: 100%|█████████▉| 499/500 [30:47<00:03,  3.70s/it]

Episode 499/500 | Avg Why: 0.6678 | Avg Inconsistency: 0.9999 | Last Entanglement: 0.990 | Last Chaoticity: 0.001


Episodes: 100%|██████████| 500/500 [30:51<00:00,  3.70s/it]

Episode 500/500 | Avg Why: 0.6653 | Avg Inconsistency: 1.0000 | Last Entanglement: 0.990 | Last Chaoticity: 0.001

Simulation complete. Generating plots and saving results...


TypeError: Object of type float32 is not JSON serializable